In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import string

from kauffman_network_helper.py import *
from theoretical_estimate_helper.py import *
from catalytic_schemes.py import *

#### Plotting Helper Functions

In [ ]:

def label_panels(axes, x=-0.08, y=1.02, fontsize=14, weight='bold'):
    """
    Add (a), (b), (c), ... to a list or array of axes.
    x,y are in axes fraction coordinates (0..1).
    """
    axes = axes.flat if hasattr(axes, "flat") else axes
    for i, ax in enumerate(axes):
        ax.text(x, y, f"{string.ascii_lowercase[i]})",
                transform=ax.transAxes, ha='left', va='bottom',
                fontsize=fontsize, fontweight=weight)

# usage

### Figure 1: Probabilistic Emergence of RAF in Simulation

Plotting Data generated from RAF_Emergence_Sim.py

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd

# --- data ---
file_path = 'Autocatalytic-Reaction-Networks/raf_results.csv'
all_runs_data = pd.read_csv(file_path)

# --- plot ---
fig, ax = plt.subplots(figsize=(7, 4.5))

# Tableau 10 palette
tab10 = plt.get_cmap('tab10').colors  # tuple of 10 RGB colors

# stable ordering so color assignment is consistent across runs
n_vals = sorted(all_runs_data.n.unique())

for i, n_val in enumerate(n_vals):
    if n_val <= 5:
        continue
    plot_df = all_runs_data[all_runs_data.n == n_val].sort_values('f')

    ax.plot(
        plot_df.f,
        plot_df.raf_prob,
        label=f"n = {n_val}, {plot_df.trial_count.iloc[0]}",
        color=tab10[i % len(tab10)],
        linewidth=2.0,
    )

# axes / styling
ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
ax.set_xlabel('f')
ax.set_ylabel('P(RAF)')

# compact legend
ax.legend(fontsize=14, frameon=False, ncol=2)

fig.tight_layout()
plt.show()


### Figure 3: RAF Emergence can be estimated by the complementary CDF of teh distribution of total catalysts

##### Producing Data for Fig 3a: Growth of $B_k$ and $S \choose k$

In [ ]:
n = 6

# Size of irrRAF core set chosen to be proportional to R as proven in paper
Q_new = np.sqrt(n * 2**n) * 500


X, F, R = create_XFR(n)
mR = len(X) * len(R) / 2
k_estimate = (2**(n+1) - 2) * f_linear_fit(n)
k_values = np.arange(int(k_estimate * 0.9), int(k_estimate * 1.1))


conditional_probability = [
        np.exp(
            min(
                log_sum(raf_term(i, mR, Q_new, k_prime=(2**(n+1)-2)*f_linear_fit(n), n=n))
                - wiki_approx(mR, i),
                0
            )
        )
        for i in k_values
    ]


k_prime = k_values[np.argmax(conditional_probability)]


b_k = [log_sum(raf_term(i, mR, Q_new, k_prime=(2**(n+1)-2)*f_linear_fit(n), n=n)) for i in k_values]
binom_term = [wiki_approx(mR, i) for i in k_values]


##### Producing Data for 3b: Distribution of Catalytic Pairs

In [ ]:
N, n = 1000, 6
X, F, R = create_XFR(n)
X_copy = X.copy()
F_copy = F.copy()
R_copy = R.copy()
fs_dist = [0.5, 1.25, 2]
cols = ['#1B4F72', '#2E86C1', '#85C1E9']

color_idx = 0

uniform_distributions = {}

for i,f in enumerate(fs_dist):
    
    counts, raf_count = [], 0
    for _ in range(N):
        X, F, R = X_copy.copy(), F_copy.copy(), R_copy.copy()
        C = create_uniform_catalysts(X, len(R), f)
        #raf_count += RAF(F, R, C)
        counts.append(sum(len(v) for v in C.values())/2)

  

    uniform_distributions[f] = [counts] 

##### Producing Data for 3c: Alignment of Theory and Simulation Curves

In [ ]:
N, n = 500, 6
X, F, R = create_XFR(n)
X_copy = X.copy()
F_copy = F.copy()
R_copy = R.copy()
fs = np.linspace(0, 2, 9)#[0.25, 0.5, 0.75, 1.25,1.5, 2]

rafs = []

for i,f in enumerate(fs):
    counts, raf_count = [], 0
    for _ in range(N):
        X, F, R = X_copy.copy(), F_copy.copy(), R_copy.copy()
        C = create_uniform_catalysts(X, len(R), f)
        raf_count += RAF(F, R, C)
        #counts.append(sum(len(v) for v in C.values())/2)
    rafs.append(raf_count / N)


f_theory = np.linspace(0,2, 100)
theory_curve = [1 - uniformCatalysis(n,f).cdf(k_prime-1) for f in f_theory]

##### Generating Plot

In [ ]:


fs = np.linspace(0, 2, 9)

# --- COLORS -------------------------------------------------------------
cols = ['#a9cce3', '#80a9c9', '#5787af', '#2d6495', '#123b66']
k_prime_color = '#9E2A2B'
sim_color     = cols[-1]
theory_color  = '#FFC107'

b_k_color     = cols[0]
binom_color   = cols[-1]
cond_prob_color = cols[-1]

# --- FIGURE + GRID ------------------------------------------------------
fig = plt.figure(figsize=(12, 6), constrained_layout=True)

# Grid: 2 rows × 2 columns
# Left column: stacked panels (top-left & bottom-left)
# Right column: one tall panel (theory)
gs = gridspec.GridSpec(
    2, 2, figure=fig,
    width_ratios=[1.4, 1.1],    # right column wider
    height_ratios=[1.0, 1.0],   # two equal-height left panels
    wspace=0.18, hspace=0.3
)

# Left column stacked
ax_main = fig.add_subplot(gs[0, 0])    # top-left  → (a)
ax2     = fig.add_subplot(gs[1, 0])    # bottom-left → (b)
# Right column spans both rows → (c)
ax3     = fig.add_subplot(gs[:, 1])

# --- PLOTTING -----------------------------------------------------------

# Panel (a): conditional probability
ax_main.scatter(k_values, conditional_probability, color=cond_prob_color)
ax_main.axvline(k_prime, ls="--", lw=2, color=k_prime_color, label='$k_{prime} = 151$')
ax_main.set_xlabel('k')

# inset
ax_inset = ax_main.inset_axes([0.65, 0.25, 0.4, 0.55])
ax_inset.plot(k_values, b_k, label='|$B_k$|', color=b_k_color)
ax_inset.plot(k_values, binom_term, label=r'$\binom{S}{k}$', color=binom_color)
ax_inset.axvline(k_prime, color=k_prime_color, lw=1, ls="--")
ax_inset.legend(fontsize=9, frameon=False, loc='upper left')
ax_inset.set_xlabel('k')
ax_inset.set_ylabel('Log Counts')

# Panel (b): distributions
for i, f in enumerate(fs_dist):
    if f < 1:
        bin_count = 15
    else:
        bin_count = 30

    ax2.hist(uniform_distributions[f], bins=bin_count, alpha=0.6,
             label=f"f = {f}", density=True, color=cols[i*2])
    unif = uniformCatalysis(n, f)
    scaling = 10 if unif.x_min < 48 else 0
    xs = np.arange(int(unif.x_min) - scaling, int(unif.x_max) + scaling, 1)
    ax2.plot(xs, unif.pdf(xs), lw=2, color=cols[i*2])
ax2.axvline(k_prime, ls="--", lw=2, color=k_prime_color)
ax2.set_xlabel('k')
ax2.set_ylabel('Frequency')

# Panel (c): theory vs simulation
ax3.scatter(fs, rafs, marker="o", lw=2, color=sim_color, label='Simulation')
ax3.plot(f_theory, theory_curve, lw=2.5, ls="--",
         color=theory_color, label='Theory')
ax3.set_xlabel('f')
ax3.set_ylabel('P(RAF)')

# --- FORMATTING ---------------------------------------------------------
ax_main.set_xlabel('k')
ax_main.set_ylabel('P(RAF|k)')
ax_main.legend(fontsize=12, frameon=False, loc='upper left')
ax2.legend(fontsize=12, frameon=False, loc='upper right')
ax3.legend(fontsize=12, frameon=False, loc='best')

# panel labels (a, b, c)
label_panels(np.array([ax_main, ax2, ax3]))

# --- SAVE / SHOW --------------------------------------------------------
plt.tight_layout()
fig.savefig("Fig3.pdf",
            bbox_inches='tight', pad_inches=0.02)
plt.show()


### Figure 4: Theoretical model explains RAF emergence across catalytic schemes

##### Generating Simulation Data for Each Catalysis Models

In [ ]:
n = 6

df_data = pd.DataFrame(columns= ['method', 'f','batch_id', 'success', 'total_runs'])
df_data.to_csv(f'n_{n}_sims.csv', index= False)

N = 5
reps = 100

uniform_distributions_2 = {}
plaw_distributions_2 = {}
allornone_distributions_2 = {}
sparse_distributions_2 = {}

for i in range(N):
    print(i+1)
    
    fs = np.linspace(0,2.5,11)

    uniform_output = []
    
    for f in tqdm(fs):
        counts = []
        raf_count = 0
        for j in range(reps):
            X,F,R= create_XFR(n)
            C = create_uniform_catalysts(X, len(R),f)
            raf_count += RAF(F,R,C)
            counts.append(sum([len(c)/2 for c in C.values()]))

        uniform_distributions_2[f] = counts
        uniform_output.append(raf_count)


    update_df('Uniform', reps, fs,uniform_output,n=n )

    ######################################

    n = 6
 

    power_output = []
   
    for f in tqdm(fs):
        counts = []
        raf_count = 0
        plaw = powerlawCatalysis(n, f)
        for j in range(reps):
            X,F,R= create_XFR(n)
            C = create_powerlaw_catalysts(X, len(R),plaw.s)
            counts.append(sum([len(c)/2 for c in C.values()]))

            raf_count += RAF(F,R,C)
        #output.append([n, f, raf_count/reps])
        plaw_distributions_2[f] = counts
        power_output.append(raf_count)


    update_df('Power', reps, fs,power_output,n=n )

    #######################################
    n = 6
    #reps = 100
    

    allornone_output = []
    
    for f in tqdm(fs):
        counts = []
        raf_count = 0
        for j in range(reps):
            X,F,R= create_XFR(n)
            #p = f/len(R)
            C = create_allornone_catalysts(X, len(R),f)
            counts.append(sum([len(c)/2 for c in C.values()]))

            raf_count += RAF(F,R,C)
        #output.append([n, f, raf_count/reps])
        allornone_distributions_2[f] = counts
        allornone_output.append(raf_count)

    #print(raf_count/reps)
    update_df('All or None', reps, fs,allornone_output,n=n )

    ########################################

    n = 6


    sparse_output = []
    
    for f in tqdm(fs):
        counts = []
        raf_count = 0
        for j in range(reps):
            #p = f/len(R)
            X,F,R= create_XFR(n)
            C = create_sparse_catalysts(X, len(R),f,n)
            counts.append(sum([len(c)/2 for c in C.values()]))
            raf_count += RAF(F,R,C)
        #output.append([n, f, raf_count/reps])
        sparse_distributions_2[f] = counts
        sparse_output.append(raf_count)

    #print(raf_count/reps)
    update_df('Sparse', reps, fs, sparse_output,n=n )


##### Producing Theoretical Curves for Each Catalysis Model

In [ ]:
n =6
X,F,R = create_XFR(n)
mR = len(R) * len(X)/2
max_it = int(mR * 6/len(R))



f_theoretical = np.linspace(0, 2.5, 50)

colors = ['#6B8E23', '#FFA500', '#ADD8E6', '#DC143C']
k_values = [i for i in range(1, max_it)]
conditional_probability = [np.exp(min(log_sum(raf_term(i, mR, Q_new,k_prime = (2**(n+1) -2) *f_linear_fit(n),  n= n)) - wiki_approx(mR, i), 0)) for i in k_values]
k_prime = k_values[conditional_probability.index(1)]
#k_prime = k_values[conditional_probability.index(1)]


uniform_theory = [1- uniformCatalysis(n, f).cdf(k_prime) for f in f_theoretical]
plaw_theory = [1- powerlawCatalysis(n, f).cdf(k_prime) for f in f_theoretical]
allornone_theory = [1- AllOrNoneCatalysis(n, f).cdf(k_prime) for f in f_theoretical]
sparse_theory = [1- SparseCatalysis(n, f).cdf(k_prime) for f in f_theoretical]


##### Producing Figure

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import string
from matplotlib.patches import Patch

# --- Color palette ----------------------------------------------------
cols = ['#A9CCE3', '#80A9C9', '#5787AF', '#2D6495', '#123B66']

schemes =  {
    "Uniform":   "#C13BAF",  # deep turquoise
    "PowerLaw":  "maroon", #A02C2C",  # dark crimson
    "Sparse":    "#6A3D9A",  # vibrant purple
    "AllOrNone": "#009E73",  # teal green
}


# same palette for every inset
f_palette = {
    0.5:  '#A9CCE3',   # light blue
    1.25: '#5499C7',   # mid blue
    2.0:  '#123B66',   # dark navy
}

f_sets = {
    "Uniform":    [0.5, 1.25, 2.0],
    "PowerLaw":   [1.25, 2],
    "AllOrNone":  [0.5, 2.0],
    "Sparse":     [1.25],
}


sim_color     = cols[-1]
theory_color  = '#FFC107'
k_prime_color = '#9E2A2B'
dist_fill     = cols[0]


# --- Figure layout ----------------------------------------------------
fig = plt.figure(figsize=(12, 6), constrained_layout=True)
gs = gridspec.GridSpec(2, 2, figure=fig, wspace=0.3, hspace=0.25)

axes = []
for i in range(4):
    row, col = divmod(i, 2)
    ax = fig.add_subplot(gs[row, col])
    axes.append(ax)


df_data = pd.read_csv('n_6_sims.csv')
#df_data =df_data[df_data.total_runs > 100]

stats_df = calculate_error_bars(df_data)


methods = ['Uniform', 'Power', 'All or None', 'Sparse']



# for i,m in enumerate(methods):
#     df = stats_df[stats_df.method == m]
    
#     ax1.errorbar(df.f, df.mean_success_rate, 
#         yerr=[df.std_error.values * 1.96, df.std_error.values *1.96],color= colors[i], fmt ='o',capsize=5,)


########### Uniform ###########
#axes[0].plot(fs, y_sim, color=sim_color, lw=2, label="Simulation")
df = stats_df[stats_df.method == 'Uniform']
axes[0].errorbar(df.f, df.mean_success_rate, yerr=[df.std_error.values * 1.96, df.std_error.values *1.96],color= schemes["Uniform"], fmt ='o',capsize=5, label = 'Uniform' )
axes[0].plot(f_theoretical, uniform_theory, color=theory_color, lw=2, ls="--", label="Theory")


###

axes[0].set_xlabel("f")
axes[0].set_ylabel("P(RAF)")
#ax.set_title(f"Uniform", fontsize=10, pad=2)
axes[0].margins(x=0.02, y=0.1)
axes[0].legend(fontsize=10, frameon=False, loc="best")

# Inset distribution
ax_inset = axes[0].inset_axes([0.65, 0.2, 0.45, 0.3])
for i,f in enumerate(f_sets['Uniform']):
    bins = np.concatenate(([ -0.5, 0.5 ], np.arange(1.5, max(uniform_distributions_2[f])+1.5, 5)))
    ax_inset.hist(uniform_distributions_2[f], bins=bins, alpha=0.6, label=f"f = {f}", density= True, color = f_palette[f])
    unif = uniformCatalysis(n,f)
    if unif.x_min < 48:
        scaling = 10
    else:
        scaling = 0
    
    xs = np.arange(int(unif.x_min) - scaling, min(int(unif.x_max)+ scaling, 700) ,1)

    ax_inset.plot(xs, unif.pdf(xs) , lw=2,color= f_palette[f])
    
ax_inset.set_xlim(0, 325)
#ax_inset.legend()

# Optional threshold marker in inset
ax_inset.axvline(k_prime, color=k_prime_color, ls="--", lw=2, label = '$k_{prime}$')
ax_inset.set_xlabel("k",labelpad=-5)
ax_inset.set_ylabel("Frequency")

# Panel letter (a,b,c,d)
axes[0].text(-0.08, 1.03, f"a)",
        transform=axes[0].transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)



######### Power Law #######
df = stats_df[stats_df.method == 'Power']
axes[1].errorbar(df.f, df.mean_success_rate, yerr=[df.std_error.values * 1.96, df.std_error.values *1.96],color= schemes["PowerLaw"], fmt ='o',capsize=5, label = 'Power Law' )
axes[1].plot(f_theoretical, plaw_theory, color=theory_color, lw=2, ls="--", label="Theory")


###

axes[1].set_xlabel("f")
axes[1].set_ylabel("P(RAF)")
#ax.set_title(f"Uniform", fontsize=10, pad=2)
axes[1].margins(x=0.02, y=0.1)
axes[1].legend(fontsize=10, frameon=False, loc="best")

# Inset distribution
ax_inset = axes[1].inset_axes([0.65, 0.2, 0.45, 0.3])
for i,f in enumerate(f_sets['PowerLaw']):
    #bins = np.concatenate(([ -0.5, 0.5 ], np.arange(1.5, max(plaw_distributions_2[f])+1.5, 5)))
    ax_inset.hist(plaw_distributions_2[f], bins=25, alpha=0.6, label=f"f = {f}", density= True, color = f_palette[f])
    plaw = powerlawCatalysis(n,f)
    if plaw.x_min < 48:
        scaling = 10
    else:
        scaling = 0
    
    xs = np.arange(0, 500 ,1)

    ax_inset.plot(xs, [plaw.pdf(x) for x in xs] , lw=2,color= f_palette[f])
    

# Optional threshold marker in inset
ax_inset.axvline(k_prime, color=k_prime_color, ls="--", lw=2, label = '$k_{prime}$')
ax_inset.set_xlim(0, 500)
ax_inset.set_xlabel("k",labelpad=-5)
ax_inset.set_ylabel("Frequency")
#ax_inset.legend()
#ax_inset.set_ylim(0, 0.5)
# Panel letter (a,b,c,d)
axes[1].text(-0.08, 1.03, f"b)",
        transform=axes[1].transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)



######### All or None #######
df = stats_df[stats_df.method == 'All or None']
axes[2].errorbar(df.f, df.mean_success_rate, yerr=[df.std_error.values * 1.96, df.std_error.values *1.96],color= schemes["AllOrNone"], fmt ='o',capsize=5, label = 'All or None' )
axes[2].plot(f_theoretical, allornone_theory, color=theory_color, lw=2, ls="--", label="Theory")


###

axes[2].set_xlabel("f")
axes[2].set_ylabel("P(RAF)")
#ax.set_title(f"Uniform", fontsize=10, pad=2)
axes[2].margins(x=0.02, y=0.1)
axes[2].legend(fontsize=10, frameon=False, loc="best")

# Inset distribution
ax_inset = axes[2].inset_axes([0.2, 0.6, 0.45, 0.3])
for i,f in enumerate(f_sets['AllOrNone']):
    #bins = np.concatenate(([ -0.5, 0.5 ], np.arange(1.5, max(allornone_distributions_2[f])+1.5, 5)))
    ax_inset.hist(allornone_distributions_2[f], bins=30, alpha=0.6, label=f"f = {f}", density= True, color = f_palette[f])
    allornone = AllOrNoneCatalysis(n,f)
    if allornone.x_min < 48:
        scaling = 10
    else:
        scaling = 0
    
    #xs = np.arange(max(int(allornone.x_min) - scaling,0), min(int(allornone.x_max)+ scaling, 700) ,1)

    #ax_inset.plot(xs, [allornone.pdf(x) for x in xs] , lw=3,color= f_palette[f])
    

# Optional threshold marker in inset
ax_inset.axvline(k_prime, color=k_prime_color, ls="--", lw=2, label = '$k_{prime}$')
ax_inset.set_xlim(-50, 1050)
ax_inset.set_xlabel("k", labelpad=0)
ax_inset.set_ylabel("Frequency")
#ax_inset.legend()

# Panel letter (a,b,c,d)
axes[2].text(-0.08, 1.1, f"c)",
        transform=axes[2].transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)
axes[2].set_ylim(0,1)


######### Sparse #######
df = stats_df[stats_df.method == 'Sparse']
axes[3].errorbar(df.f, df.mean_success_rate, yerr=[df.std_error.values * 1.96, df.std_error.values *1.96],color= schemes["Sparse"], fmt ='o',capsize=5, label = 'Sparse' )
axes[3].plot(f_theoretical, sparse_theory, color=theory_color, lw=2, ls="--", label="Theory")


###

axes[3].set_xlabel("f")
axes[3].set_ylabel("P(RAF)")
#ax.set_title(f"Uniform", fontsize=10, pad=2)
axes[3].margins(x=0.02, y=0.1)
axes[3].legend(fontsize=10, frameon=False, loc="best")



# Inset distribution
ax_inset = axes[3].inset_axes([0.6, 0.2, 0.45, 0.3])
for i,f in enumerate(f_sets['Sparse']):
    bins = np.concatenate(([ -0.5, 0.5 ], np.arange(1.5, max(sparse_distributions_2[f])+1.5, 5)))
    ax_inset.hist(sparse_distributions_2[f], bins=bins, alpha=0.6, label=f"f = {f}", density= True, color = f_palette[f])
    sparse = SparseCatalysis(n,f)
    if sparse.x_min < 48:
        scaling = 10
    else:
        scaling = 0
    
    xs = np.arange(max(int(sparse.x_min) - scaling,0), min(int(sparse.x_max)+ scaling,500) ,1)

    ax_inset.plot(xs, [sparse.pdf(x) for x in xs] , lw=2,color= f_palette[f])
    

ax_inset.set_xlim(-20, 500)
#ax_inset.legend()
#ax_inset.set_ylim(0, 0.2)
# Optional threshold marker in inset
ax_inset.axvline(k_prime, color=k_prime_color, ls="--", lw=2, label = '$k_{prime}$')
ax_inset.set_xlabel("k",labelpad=0)
ax_inset.set_ylabel("Frequency")

# Panel letter (a,b,c,d)
axes[3].text(-0.08, 1.1, f"d)",
        transform=axes[3].transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)

# --- Main plotting loop -----------------------------------------------
# for i, ax in enumerate(axes):
#     # Main plot
#     ax.plot(x, y_sim, color=sim_color, lw=2, label="Simulation")
#     ax.plot(x, y_theory, color=theory_color, lw=2.5, ls="--", label="Theory")
#     ax.axvline(5, color=k_prime_color, lw=1.5, ls="--")

#     ax.set_xlabel("f")
#     ax.set_ylabel("P(RAF)")
#     ax.set_title(f"Condition {i+1}", fontsize=10, pad=2)
#     ax.margins(x=0.02, y=0.1)
#     ax.legend(fontsize=7, frameon=False, loc="best")

#     # Inset distribution
#     ax_inset = ax.inset_axes([0.6, 0.15, 0.35, 0.35])
#     ax_inset.fill_between(dist_x, dist_y, color=dist_fill, alpha=0.5)
#     ax_inset.plot(dist_x, dist_y, color=sim_color, lw=1.5)
#     ax_inset.set_xticks([]); ax_inset.set_yticks([])
#     ax_inset.set_xlim(0, 1); ax_inset.set_ylim(0, 1.1 * dist_y.max())

#     # Optional threshold marker in inset
#     ax_inset.axvline(0.5, color=k_prime_color, ls="--", lw=1)

#     # Panel letter (a,b,c,d)
#     ax.text(-0.08, 1.03, f"({string.ascii_lowercase[i]})",
#             transform=ax.transAxes, ha='left', va='bottom',
#             fontweight='bold', fontsize=10)

# --- Fine-tuning ------------------------------------------------------
# for ax in axes:
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.tick_params(direction='out', length=3, width=0.8)

# fig.suptitle("Theory–Simulation Behavior Across Catalytic Schemes",
#              fontsize=12, fontweight='bold', y=1.02)

handles = [Patch(facecolor=c, edgecolor='none', alpha=0.6, label=f"f = {f}")
           for f, c in f_palette.items()] + [Patch(facecolor=k_prime_color, edgecolor='none', alpha=1, label="$k_{prime}$")]

# figure-level legend (outside, bottom center)
fig.legend(handles=handles,
           loc='lower center', ncol=len(handles),
           frameon=False, title="",
           #bbox_to_anchor=(0.5, -0.02)
           # )  # adjust if needed
)
# --- Save / Show ------------------------------------------------------
plt.tight_layout()
fig.savefig("Paper/Figures/Fig2/Fig2.pdf",
            bbox_inches='tight', pad_inches=0.02)
plt.show()

### Figure 5: Transition Sharpness can be explained by the derivative of the CDF

##### Computing Derivatives of Each Distribution

In [ ]:
def binom_cdf_derivative(k_prime, S, f, len_R):
    return len_X* binom.pmf(k_prime, S -1, 2*f/len_R)


# Finite Difference Derivative
def powerlaw_empirical_derivative(k_prime, f,n, h= 1e-3):
    f_hp = powerlawCatalysis(n,f+h).cdf(k_prime)
    f_hm = powerlawCatalysis(n,f-h).cdf(k_prime)

    D_h = (f_hp - f_hm)/(2*h)

    return D_h

def sparse_derivative(k_prime, S, f, len_R, len_X, n ):
    p = 2*f*n/len_R
    res = 2 * n * len_X / len_R * sum([
    binom.cdf(k_prime, int(i * len_R / 2), 1 / n) * (
        (math.comb(len_X - 1, i - 1) * p**(i - 1) * (1 - p)**(len_X - i) if i > 0 else 0)
        - math.comb(len_X - 1, i) * p**i * (1 - p)**(len_X - i - 1)
    )
    for i in range(len_X+1)
])
    return -res
    #eturn 2* S/len_R * -(math.comb(int(S-1), k_prime) * (2*f / len_R)**k_prime  * (1 - 2*f / len_R)**(S-1 -k) )

def allornone_derivative(f, len_X, len_R ):
    p = 2*f / len_R
    return  len_X * (1-p)**(len_X -1) *2/len_R

In [ ]:
schemes =  {
    "Uniform":   "#C13BAF",  # deep turquoise
    "PowerLaw":  "maroon", #A02C2C",  # dark crimson
    "Sparse":    "#6A3D9A",  # vibrant purple
    "AllOrNone": "#009E73",  # teal green
}
f_sharp = np.linspace(0,3,100)


fig, axs = plt.subplots(2, 2, figsize=(8, 5))
(ax1, ax2), (ax3, ax4) = axs

ax1.plot(f_sharp, [len_X* binom.pmf(k_prime, mR -1, 2*f/len_R) for f in f_sharp], label = 'Uniform', color = schemes['Uniform'], lw = 2)
ax1.set_xlabel('f')
ax1.set_ylabel('dP(RAF)/df')
ax1.legend(fontsize = 15)

ax1.text(-0.2, 0.9, f"a)",
        transform=ax1.transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)

ax2.plot(f_sharp, [- powerlaw_empirical_derivative(k_prime, f, n) for f in f_sharp], label = 'Power Law',color = schemes['PowerLaw'], lw =2)
ax2.set_xlabel('f')
ax2.set_ylabel('dP(RAF)/df')
ax2.legend(fontsize = 15)

ax2.text(-0.2, 0.9, f"b)",
        transform=ax2.transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)

ax3.plot(f_sharp, [ allornone_derivative(f, len_X, len_R) for f in f_sharp], label = 'All or None',color = schemes['AllOrNone'], lw = 2)
ax3.set_xlabel('f')
ax3.set_ylabel('dP(RAF)/df')
ax3.legend(fontsize = 15)

ax3.text(-0.2, 1.0, f"c)",
        transform=ax3.transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)

ax4.plot(f_sharp, [sparse_derivative(k_prime, mR, f, len_R, len_X, n) for f in f_sharp], label = 'Sparse', color = schemes['Sparse'], lw = 2)
ax4.set_xlabel('f')
ax4.set_ylabel('dP(RAF)/df')
ax4.legend(fontsize = 15)

ax4.text(-0.2, 1.0, f"d)",
        transform=ax4.transAxes, ha='left', va='bottom',
        fontweight='bold', fontsize=16)

plt.tight_layout()
fig.savefig("Paper/Figures/Fig3/Fig3.pdf",
            bbox_inches='tight', pad_inches=0.02)
plt.show()